In [1]:
import re
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic

In [2]:
# 데이터 로드
data = pd.read_csv("/Users/naeunyoon/Desktop/Project/Project/sentiment/predicted_sentiment_scores_v4.csv")
data['price'] = data['price'].replace('가격 정보 없음', np.nan)
data['price'] = data['price'].str.replace(',', '', regex=False).astype(float)
data['sentiment_score'] = pd.to_numeric(data['predicted_score'], errors='coerce')
data

/var/folders/wj/15t2wl4s1k9974lyhp_nz7pc0000gn/T/ipykernel_40835/3679226212.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/Users/naeunyoon/Desktop/Project/Project/sentiment/predicted_sentiment_scores_v4.csv")


,hotel,star,review,date,price,address,district,length,clean_review,latitude,longitude,predicted_score,sentiment_score
0,유성 메이호텔,5,매우 만족합니다 깨끗하고 시설도좋고 주변에 먹을것도많고 주차자리도 매우 널하고 1층...,2024.10.18,NaN,대전 유성구 봉명동 551-17,유성구,250,매우 만족합니다 깨끗하고 시설도 좋고 주변에 먹을 것도 많고 주차 자리도 매우 널하...,36.354776,127.342764,0.984308,0.984308
1,유성 메이호텔,4,주변에 숙소가 많은데 다른숙소를 주로 이용하다 이번에 처음 이용해보았는데 객실도 너...,2024.10.18,NaN,대전 유성구 봉명동 551-17,유성구,262,주변에 숙소가 많은데 다른 숙소를 주로 이용하다 이번에 처음 이용해 보았는데 객실도...,36.354776,127.342764,0.985063,0.985063
2,유성 메이호텔,5,봉명동 술집에서 술마실때 위치가 너무 좋아요 항상 찾는곳으로 주차장도 지상도 있고 ...,2024.10.28,NaN,대전 유성구 봉명동 551-17,유성구,255,봉명동 술집에서 술 마실 때 위치가 너무 좋아요 항상 찾는 곳으로 주차장도 지상도 ...,36.354776,127.342764,0.985439,0.985439
3,유성 메이호텔,5,짐 놓고갔엇는데 프론트 분 이 되게 친절하셨었어요 아 송주차장이 없지만 그래도 주변...,2024.10.29,NaN,대전 유성구 봉명동 551-17,유성구,192,짐 놓고 갔었는데 프런트 분 이 되게 친절하셨었어요 아 송 주차장이 없지만 그래도 ...,36.354776,127.342764,0.985395,0.985395
4,유성 메이호텔,4,할머니 댁같은 느낌객실깔끔한데 오래된 느낌은 못 지우겠어요 전담을 옆방에서 피는지 ...,2024.11.05,NaN,대전 유성구 봉명동 551-17,유성구,259,할머니 댁 같은 느낌 객실 깔끔한데 오래된 느낌은 못 지우겠어요 전담을 옆방에서 피...,36.354776,127.342764,0.802685,0.802685
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233530,대전 유성 발렌타인,3,안마방 싸고 젛음,2017.10.20,20000.0,대전광역시 유성구 장대로 20-19,유성구,9,안마방 싸고 젛음,36.357106,127.337561,0.982479,0.982479
233531,대전 유성 발렌타인,5,합리적인가격에 편히 깔끔하게있다갑니다,2017.10.20,20000.0,대전광역시 유성구 장대로 20-19,유성구,20,합리적인 가격에 편히 깔끔하게 있다 갑니다,36.357106,127.337561,0.983028,0.983028
233532,대전 유성 발렌타인,4,50원 할인되서 되게싸게 갔어요가격대비 편안하고 좋았습니당,2017.10.18,20000.0,대전광역시 유성구 장대로 20-19,유성구,35,50원 할인돼서 되게 싸게 갔어요 가격 대비 편안하고 좋았습니다,36.357106,127.337561,0.985431,0.985431
233533,대전 유성 발렌타인,3,잘 머물다왔습니다,2017.10.18,20000.0,대전광역시 유성구 장대로 20-19,유성구,9,잘 머물다 왔습니다,36.357106,127.337561,0.977383,0.977383


In [3]:
# 대전 행정구역, 법정동 위경도 데이터 로드
daejeon = pd.read_csv("/Users/naeunyoon/Desktop/Project/dajeon_coordinates.csv")
daejeon

,District,Area,Latitude,Longitude
0,동구,중앙동,36.331124,127.435883
1,동구,신인동,36.322583,127.441457
2,동구,효동,36.310265,127.456502
3,동구,판암1동,36.317251,127.469372
4,동구,판암2동,36.320870,127.454565
...,...,...,...,...
77,대덕구,법2동,36.374268,127.437356
78,대덕구,신탄진동,36.440321,127.461613
79,대덕구,석봉동,36.450871,127.424234
80,대덕구,덕암동,36.431049,127.428512


In [4]:
# 호텔별 평균 값 계산
hotel_grouped = data.groupby("hotel").agg({
    "star": "mean",  # 별점의 평균
    "sentiment_score": "mean",  # 감성 점수의 평균
    "price": "mean",  # 가격의 평균
    "latitude": "first",  # 동일 호텔의 위도
    "longitude": "first",  # 동일 호텔의 경도
    "review": "count",  # 리뷰 개수
    "address": "first"
}).reset_index()

hotel_grouped.head(10)

,hotel,star,sentiment_score,price,latitude,longitude,review,address
0,경하온천호텔,4.253086,0.920392,72666.666667,36.355720,127.349743,324,대전 유성구 봉명동 540-16
1,고층,5.000000,0.983972,183000.000000,36.353139,127.345844,25,대전 유성구 봉명동 548-6
2,굿모닝 레지던스 호텔 휴,4.403226,0.923777,NaN,36.352291,127.381679,372,대전 서구 둔산동 1314
3,굿모닝 레지던스호텔 휴,4.535332,0.924661,145000.000000,36.352291,127.381679,467,대전광역시 서구 둔산로73번길 21
4,대덕 리치,4.223958,0.856797,30000.000000,36.360405,127.424781,384,대전 대덕구 중리동 401-22
5,대전 BnB,4.344934,0.863926,NaN,36.325973,127.428775,1293,대전 중구 대흥동 133
6,대전 Boutique 호텔 이틀,4.677989,0.948740,NaN,36.353344,127.438416,736,대전 동구 용전동 66-4
7,대전 CITYHOTEL 시애틀,4.615970,0.928506,NaN,36.351729,127.435673,1052,대전 동구 용전동 63-8
8,대전 HI 호텔,3.527273,0.771352,45000.000000,36.328081,127.461223,55,대전 동구 용운동 296-6
9,대전 HOTEL BON,4.499743,0.913590,NaN,36.352815,127.435882,1943,대전 동구 용전동 64-21


In [5]:
# 형태소 분석기 초기화
okt = Okt()

# 지역 추출 함수 (동/구 포함한 지역명 추출)
def extract_location(text):
    tokens = okt.nouns(text)  # 명사 추출
    locations = [word for word in tokens if '동' in word or '구' in word]  # '동' 또는 '구'가 포함된 명사만 추출
    return locations

# 대전 내에서 지역을 찾는 함수 (동/구를 제외한 지역명을 기준으로 찾기)
def find_location_in_daejeon(locations, daejeon):
    found_locations = []
    for loc in locations:
        if '구' in loc:
            clean_loc = loc.rstrip('구')  # '구'를 제외한 지역명 추출
            found_locations += [f"{district} {area}" for district, area in zip(daejeon["District"], daejeon["Area"]) if clean_loc in area]
        elif '동' in loc:
            clean_loc = loc.rstrip('동')  # '동'을 제외한 지역명 추출
            found_locations += [f"{district} {area}" for district, area in zip(daejeon["District"], daejeon["Area"]) if clean_loc in area]
    return found_locations

In [6]:
# 사용자 요청 처리 함수
def process_user_request(content, daejeon, default_location=(36.3504, 127.3845)):  # 기본 위치 설정
    # 지역 추출
    locations = extract_location(content)
    print("추출된 지역:", locations)

    # 대전 내에서 위치 찾기
    user_location = find_location_in_daejeon(locations, daejeon)
    print("대전 내에서 찾은 지역:", user_location)

    if user_location:
        # 대전 지역이 있는 경우, 첫 번째 매칭된 지역을 사용
        user_area = user_location[0]  # 첫 번째 지역 선택
        matching_rows = daejeon[daejeon["District"] + " " + daejeon["Area"] == user_area]

        if not matching_rows.empty:
            # 해당 지역이 존재하면 위치 반환
            location_row = matching_rows.iloc[0]
            user_location_coordinates = (location_row["Latitude"], location_row["Longitude"])
        else:
            # 지역이 없으면 기본 위치 사용
            user_location_coordinates = default_location
            print(f"대전 내에서 '{user_area}' 지역을 찾을 수 없어 기본 위치를 사용.")
    else:
        # 추출된 지역이 없으면 기본 위치 사용
        user_location_coordinates = default_location
        print("추출된 지역이 없으므로 기본 위치를 사용.")

    print(f"사용자가 요청한 위치: {user_location_coordinates}")
    return user_location_coordinates

In [12]:
# 예시로 사용자가 요청한 텍스트
content = "용전동 호텔 추천해줘"

# 사용자 요청에 따른 위치 설정
default_location = (36.3504, 127.3845)
user_location = process_user_request(content=content, daejeon=daejeon, default_location=default_location)

# 거리 계산 (사용자 위치와 호텔 위치 비교)
hotel_grouped["distance"] = hotel_grouped.apply(
    lambda row: geodesic(user_location, (row["latitude"], row["longitude"])).km, axis=1
)

추출된 지역: ['용전동']
대전 내에서 찾은 지역: ['동구 용전동']
사용자가 요청한 위치: (36.353219, 127.4341323)


In [13]:
# 가까운 호텔 상위 10개 출력
nearby_hotels = hotel_grouped.sort_values(by="distance").head(10)
print("추천 호텔 10개:")
print(nearby_hotels[["hotel", "distance", "star", "sentiment_score", "address"]])

추천 호텔 10개:
                         hotel  distance      star  sentiment_score  \
140                     대전 피아노  0.161458  4.512905         0.916732   
9                 대전 HOTEL BON  0.163303  4.499743         0.913590   
168      더휴식 타임터미널 호텔 용전 by 아늑  0.167850  4.607143         0.952251   
229          용전 타임터미널 호텔 by 아늑  0.167850  4.763052         0.963607   
228            용전 타임터미널 호텔 2호점  0.187822  4.763158         0.938849   
167  더휴식 타임터미널 호텔 용전 2호점 by 아늑  0.187822  4.666667         0.928269   
81       대전 용전 타임터미널 BY 아늑 2호점  0.187822  4.894737         0.955335   
78             대전 용전 HOTEL 시애틀  0.215559  4.586408         0.908833   
7             대전 CITYHOTEL 시애틀  0.215559  4.615970         0.928506   
33                      대전 두바이  0.286713  4.231827         0.865745   

                          address  
140               대전 동구 용전동 64-24  
9                 대전 동구 용전동 64-21  
168               대전 동구 용전동 64-36  
229               대전 동구 용전동 64-36  
228               대전 동구 용전동

In [14]:
# 정규화 및 점수 계산
scaler = MinMaxScaler()

# 거리 점수와 가격 점수 계산
hotel_grouped["distance_score"] = scaler.fit_transform(1 / (hotel_grouped["distance"] + 1).values.reshape(-1, 1))
hotel_grouped["price_score"] = scaler.fit_transform(1 / (hotel_grouped["price"] + 1).values.reshape(-1, 1))

# 가중치로 최종 점수 계산
hotel_grouped["final_score"] = (
    0.4 * hotel_grouped["distance_score"] +
    0.3 * hotel_grouped["star"] +
    0.2 * hotel_grouped["sentiment_score"] +
    0.1 * hotel_grouped["price_score"]
)

# final_score를 0~1로 정규화한 후, 1~5로 변환
hotel_grouped["final_score"] = scaler.fit_transform(hotel_grouped["final_score"].values.reshape(-1, 1))  # 0~1로 정규화
hotel_grouped["final_score"] = hotel_grouped["final_score"] * 4 + 1  # 1~5로 변환

# 상위 10개 추천 호텔 출력
recommended_hotels = hotel_grouped.sort_values("final_score", ascending=False).head(10)
print("\n추천 호텔 10개:")
print(recommended_hotels[["hotel", "final_score", "review", "distance", "address"]])


추천 호텔 10개:
                         hotel  final_score  review  distance  \
81       대전 용전 타임터미널 BY 아늑 2호점     5.000000     133  0.187822   
78             대전 용전 HOTEL 시애틀     4.699286     515  0.215559   
82                 대전 용전 호텔 박스     4.612741    2550  0.561599   
167  더휴식 타임터미널 호텔 용전 2호점 by 아늑     4.588669       6  0.187822   
168      더휴식 타임터미널 호텔 용전 by 아늑     4.525454      28  0.167850   
163         더휴식 아늑호텔 대전 용전 2호점     4.452974      23  0.608450   
79             대전 용전 더휴식 노크인호텔     4.435962    2024  0.548427   
62                       대전 솔비     4.368383     632  0.568356   
142                   대전 하이클래스     4.280838     254  0.376819   
61                        대전 샾     4.220708     421  0.572460   

                                  address  
81               대전광역시 동구 동서대로1683번길 76-9  
78           대전광역시 동구 동서대로1683번길 58 (용전동)  
82          대전광역시 동구 한밭대로 1316번길 34 (용전동)  
167                       대전 동구 용전동 64-35  
168                       대전 동구 용전동 64-36  
163     